#Long Form Question Answering &amp; Generated Content Detection

In this notebook, we are going to implement a long-form question answering and generated content detection model using natrual language processing (NLP) tools.

The dataset used in this project is a subset of [ELI5 dataset](https://huggingface.co/datasets/eli5_category).

The first step is to load the data and learn about the characteristics of data.
Then we have to do some pre-proceesing on the text data in order to make it ready to be given to a model to learn.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# install datasets library

In [2]:
!pip install torch
!pip install accelerate==0.30.0
!pip install faiss-cpu

import accelerate
print(accelerate.__version__)

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

#Load dataset

In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00


In [4]:
import torch
# device = torch.device('cpu')
device = 'cuda'

In [5]:
from datasets import load_dataset

In [6]:
dataset = load_dataset("eli5_category")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for eli5_category contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/eli5_category
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.war

Generating train split:   0%|          | 0/91772 [00:00<?, ? examples/s]

Generating validation1 split:   0%|          | 0/5446 [00:00<?, ? examples/s]

Generating validation2 split:   0%|          | 0/2375 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5411 [00:00<?, ? examples/s]

# EDA

##Let's see the structure of dataset

In [7]:


print('*'*100)
print('In the following part, you can see the structure of dataset:')
print('*'*100)
dataset

****************************************************************************************************
In the following part, you can see the structure of dataset:
****************************************************************************************************


DatasetDict({
    train: Dataset({
        features: ['q_id', 'title', 'selftext', 'category', 'subreddit', 'answers', 'title_urls', 'selftext_urls'],
        num_rows: 91772
    })
    validation1: Dataset({
        features: ['q_id', 'title', 'selftext', 'category', 'subreddit', 'answers', 'title_urls', 'selftext_urls'],
        num_rows: 5446
    })
    validation2: Dataset({
        features: ['q_id', 'title', 'selftext', 'category', 'subreddit', 'answers', 'title_urls', 'selftext_urls'],
        num_rows: 2375
    })
    test: Dataset({
        features: ['q_id', 'title', 'selftext', 'category', 'subreddit', 'answers', 'title_urls', 'selftext_urls'],
        num_rows: 5411
    })
})

##Let's see dataset information

In [8]:


import pandas as pd # I just created a dataframe of information
information_of_dataset = pd.DataFrame({'q_id': ['a string question identifier for each example, corresponding to its ID in the Pushshift.io Reddit submission dumps'],
                            'subreddit':['always explainlikeimfive, indicating which subreddit the question came from'],
                            'category':['tag of the question, the possible values are listed above.'],
                            'title':['title of the question, with URLs extracted and replaced by URL_n tokens'],
                            'title_urls':['list of the extracted URLs, the nth element of the list was replaced by URL_n'],
                            'selftext':['either an empty string or an elaboration of the question'],
                            'selftext_urls':['similar to title_urls but for self_text'],
                            'answers':['a list of answers, each answer has'],
                            'a_id':['a string answer identifier for each answer, corresponding to its ID in the Pushshift.io Reddit comments dumps.'],
                            'text':['he answer text with the URLs normalized'],
                            'score':['the number of upvotes - the number of downvotes the answer had received when the dumps were created'],
                            'text_urls': ['ists of the extracted URLs for every answer']},index=['description'])
print('In the following table, you can see dataset information. If you need to have a better view, try to use the icon on the right side')
print('*'*100)
information_of_dataset

In the following table, you can see dataset information. If you need to have a better view, try to use the icon on the right side
****************************************************************************************************


,q_id,subreddit,category,title,title_urls,selftext,selftext_urls,answers,a_id,text,score,text_urls
description,"a string question identifier for each example,...","always explainlikeimfive, indicating which sub...","tag of the question, the possible values are l...","title of the question, with URLs extracted and...","list of the extracted URLs, the nth element of...",either an empty string or an elaboration of th...,similar to title_urls but for self_text,"a list of answers, each answer has","a string answer identifier for each answer, co...",he answer text with the URLs normalized,the number of upvotes - the number of downvote...,ists of the extracted URLs for every answer


##Let's focues on train part of dataset and see the first element of raw_train_dataset

In [9]:

raw_train_dataset = dataset['train']
print('Let us see the first element of raw_train_dataset: ')
print('*'*100)
print('\n')

raw_train_dataset[0]

Let us see the first element of raw_train_dataset: 
****************************************************************************************************




{'q_id': '5lchat',
 'title': "Why there was a 'leap second' added to the end of 2016?",
 'selftext': '',
 'category': 'Other',
 'subreddit': 'explainlikeimfive',
 'answers': {'a_id': ['dbuoyxl', 'dbur7gi', 'dbuotht'],
  'text': ['the rotation of the earth is not a constant. in fact the rotation of the earth is slowing down, which means that a full day is getting slightly longer. without leap seconds our clocks would slowly drift ever so slightly out of sync with the actual day. we could deal with this by redefining how how long 1 second is, making it slightly longer so that one day is still exactly 24*60*60 seconds. but in practice that is really inconvenient for a lot of our technology which relies on very precise timing. its easier to just move us ahead one second every couple of years or so.',
   "The Earth's rotation is not regular. It varies a bit, so sometimes we add a second. We do this to ensure that noon is always going to be sometime around mid-day. If we did not add leap sec

#

## Let's create a better dictionary of dataset and remove some unnecessary columns

In [10]:


import numpy as np
def organize_dataset(dataset):
  '''
    input: dataset must be given as dataset['train'] or dataset['test']
  '''
  # Here, we just create a dictionary using the following columns
  #1)questions
  #2)self_text
  #3)categories
  #4)answers
  #5)answers_scores
  # Since other columns are unique acroos the whole dataset. If you would like to see wheater they are unique or not, you can
  # use the following commands
  print('Unique items in selftext_urls is just only: <', np.unique(dataset['selftext_urls'])[0],'> So, this column has been removed')
  print('Unique items in title_urls is just only: <', np.unique(dataset['title_urls'])[0],'> So, this column has been removed')
  print('Unique items in subreddit is just only: <', np.unique(dataset['subreddit'])[0],'> So, this column has been removed')


  questions = dataset['title']
  questions_selftext = dataset['selftext']
  categories = dataset['category']

  answers = []
  answers_scores = []
  for item in dataset:
    answers.append(item['answers']['text'])
    answers_scores.append(item['answers']['score'])

  dataset_dict = {"questions":questions, "selftext": questions_selftext, "category":categories, "answers":answers, "score":answers_scores}
  print('*'*100)
  print('Now, we have a clean datset dictionary with ',dataset_dict.keys())
  print('*'*100)
  return dataset_dict

In [11]:
# Define weighted sampling function
def weighted_sample(answers, scores, k=3):
    k = min(k, len(answers))

    if len(answers) == 0:
        return []

    total_score = sum(scores)
    probabilities = [score / total_score for score in scores]
    sampled_indices = np.random.choice(len(answers), size=k, replace=False, p=probabilities)
    return [answers[i] for i in sampled_indices]

In [12]:
train_dataset_dict = organize_dataset(dataset['train'])

Unique items in selftext_urls is just only: < url > So, this column has been removed
Unique items in title_urls is just only: < url > So, this column has been removed
Unique items in subreddit is just only: < explainlikeimfive > So, this column has been removed
****************************************************************************************************
Now, we have a clean datset dictionary with  dict_keys(['questions', 'selftext', 'category', 'answers', 'score'])
****************************************************************************************************


In [13]:
valid_dataset_dict = organize_dataset(dataset['validation1'])

Unique items in selftext_urls is just only: < url > So, this column has been removed
Unique items in title_urls is just only: < url > So, this column has been removed
Unique items in subreddit is just only: < explainlikeimfive > So, this column has been removed
****************************************************************************************************
Now, we have a clean datset dictionary with  dict_keys(['questions', 'selftext', 'category', 'answers', 'score'])
****************************************************************************************************


In [14]:
test_dataset_dict = organize_dataset(dataset['validation2'])

Unique items in selftext_urls is just only: < url > So, this column has been removed
Unique items in title_urls is just only: < url > So, this column has been removed
Unique items in subreddit is just only: < explainlikeimfive > So, this column has been removed
****************************************************************************************************
Now, we have a clean datset dictionary with  dict_keys(['questions', 'selftext', 'category', 'answers', 'score'])
****************************************************************************************************


# Retrieval-Augmented Generation (RAG) - with Bert & T5

Retrieval-Augmented Generation (RAG) is an advanced approach that combines retrieval mechanisms with generative models to produce more accurate and comprehensive responses. This method leverages a retrieval component to fetch relevant information from a large corpus and then uses a generative model to construct a coherent and informative answer.


> **Retrieval Component**:
>
> This part of the system retrieves relevant documents or passages from a large corpus based on the input query.
>
> It typically uses models like BM25, TF-IDF, or dense retrieval models such as DPR (Dense Passage Retrieval) to find the most relevant texts.


> **Generation Component**:
>
>Once relevant documents or passages are retrieved, a generative model like T5 or GPT-3 is used to synthesize the information into a coherent and comprehensive answer.
>
>The generative model takes the retrieved texts and the query as inputs to produce the final output.

## RAG

**Question-Question Similarity:**

Use question-question similarity to build the context instead of question-answer similarity.

We have tried both methods for finding the similar answers using TF-IDF and Bert but the first method did not have a good result. So lets see how Bert Embedding works for finding similar questions to build the context

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [16]:
from transformers import BertModel, BertTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm
import itertools
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [17]:
# Load pre-trained BERT model and tokenizer
model_name_bert = 'bert-base-uncased'
tokenizer_bert = BertTokenizer.from_pretrained(model_name_bert)
model_bert = BertModel.from_pretrained(model_name_bert).to(device)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [18]:
def encode_texts(model, tokenizer, texts, batch_size=16):
    model.eval()
    embeddings = []
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            tokens = tokenizer(batch, padding=True, truncation=True, return_tensors='pt').to(device)
            outputs = model(**tokens)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1).to(device)
            embeddings.append(batch_embeddings)
    return torch.cat(embeddings, dim=0)

def encode_questions(model, tokenizer, questions, batch_size=16):
    print("Start encoding questions:\n\n")
    embeddings = encode_texts(model, tokenizer, questions, batch_size)
    print("Finish encoding questions:\n~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
    return embeddings

def encode_answers(model, tokenizer, answers, batch_size=16):
    print("Start encoding answers:\n\n")
    # Flatten the list of lists to encode all answers at once
    flat_answers = [answer for sublist in answers for answer in sublist]
    flat_embeddings = encode_texts(model, tokenizer, flat_answers, batch_size)

    # Restructure the embeddings back to the original nested structure
    nested_answer_embeddings = []
    index = 0
    for sublist in answers:
        num_answers = len(sublist)
        if num_answers > 0:
            nested_answer_embeddings.append(flat_embeddings[index:index + num_answers])
            index += num_answers
        else:
            nested_answer_embeddings.append(torch.tensor([]))  # Handle empty lists

    print("Finish encoding answers:\n~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
    return nested_answer_embeddings

def save_Q_encodings(question_embeddings, question_embeddings_file='/kaggle/working/question_embeddings.npy'):
    # Save the embeddings to a file
    embeddings_file = 'question_embeddings.npy'
    np.save(question_embeddings_file, question_embeddings.cpu().numpy())

    # Convert list of tensors to list of numpy arrays for saving
#     nested_answer_embeddings_np = [emb.numpy() for emb in nested_answer_embeddings]
    # Save the nested list of embeddings to a file
#     np.save(answer_embeddings_file, nested_answer_embeddings_np.numpy())

    print("Saved Question Embeddings in file.")

def load_Q_encodings(train_question_embeddings_file='/kaggle/input/questionsembeddings/train_question_embeddings.npy', valid_question_embeddings_file='/kaggle/input/questionsembeddings/valid_question_embeddings.npy'):
    # Load the embeddings from the file
    train_questions_embeddings = torch.tensor(np.load(train_question_embeddings_file))
    valid_questions_embeddings = torch.tensor(np.load(valid_question_embeddings_file))

    # Load the nested list of embeddings from the file
#     loaded_answer_embeddings = [torch.tensor(emb) for emb in np.load('answer_embeddings.npy', allow_pickle=True)]
    print("Loaded Question Embeddings.")
    return train_questions_embeddings, valid_questions_embeddings

# Function to encode a query
def encode_query(model, tokenizer, query):
    model.eval()
    embeddings = []
    with torch.no_grad():
        tokens = tokenizer(query, padding=True, truncation=True, return_tensors='pt')
        outputs = model(**tokens)
        query_embeddings = outputs.last_hidden_state.mean(dim=1)
    return torch.cat(query_embeddings, dim=0)

def retrieve_similar_questions_bert(model, tokenizer, dataset_dict, query, query_index,
                                    question_embeddings, top_n=20, batch_size=8):
    # Encode questions in batches
    query_embedding = encode_texts(model, tokenizer, [query], batch_size=1).cpu().numpy()

    # Calculate cosine similarity
    question_embeddings_cpu = question_embeddings.cpu().numpy()
    similarities = cosine_similarity(query_embedding, question_embeddings_cpu).flatten()
    relevant_indices = np.argsort(similarities)[::-1]

    if query_index != None:
      np.delete(relevant_indices, np.where(relevant_indices==query_index)[0][0])

    relevant_indices = relevant_indices[:top_n]

    similar_questions = [dataset_dict["questions"][i] for i in relevant_indices]
    relevant_scores = [similarities[i] for i in relevant_indices]

    #keep only questions that their score is above 35% of max_score
    max_score = max(relevant_scores)
    threshold = 0.35 * max_score
    filtered_similar_questions = [q for q, s in zip(similar_questions, relevant_scores) if s >= threshold]
    filtered_relevant_scores = [s for s in relevant_scores if s >= threshold]
    filtered_relevant_indices = [i for i, s in zip(relevant_indices, relevant_scores) if s >= threshold]

    # Sort the results based on the relevant_scores
    sorted_indices = np.argsort(relevant_scores)[::-1]
    sorted_similar_questions = [similar_questions[i] for i in sorted_indices]
    sorted_relevant_scores = [relevant_scores[i] for i in sorted_indices]
    sorted_relevant_indices = [relevant_indices[i] for i in sorted_indices]

    return sorted_similar_questions, sorted_relevant_scores, sorted_relevant_indices


# retrieve top 20 similar questions
def retrieve_similar_answers_bert(model, tokenizer, dataset_dict, query, query_index, question_embeddings,
                                  top_n_answers=8, top_n_questions=5, percentage=0.75):
    _, _, similar_indices = retrieve_similar_questions_bert(model, tokenizer, train_dataset_dict, query, query_index, question_embeddings, top_n=top_n_questions, batch_size=16)

    scaler = MinMaxScaler()

    filtered_answer_indices_all = []

    # filter answers from each similar question by score threshold (top 30% for other questions)
    for idx in similar_indices:
        scores = np.array(dataset_dict['score'][idx]).reshape(-1, 1)
        normalized_scores = scaler.fit_transform(scores).flatten()
        threshold_value = 0.7 * np.max(normalized_scores)  # 70% threshold for other questions
        filtered_answer_indices = [i for i, score in enumerate(normalized_scores) if score >= threshold_value]
        filtered_answer_indices_all.append((idx, filtered_answer_indices))

    filtered_answer_indices = filtered_answer_indices[:top_n_answers]
    filtered_answer_indices_all = filtered_answer_indices_all[:top_n_answers]

    if query_index != None:
      # add answers of the current question with a 75% threshold
      scores = np.array(dataset_dict['score'][query_index]).reshape(-1, 1)
      normalized_scores = scaler.fit_transform(scores).flatten()
      threshold_value = 0
      if len(dataset_dict['score'][query_index]) > 1:
          threshold_value = 0.25 *np.max(normalized_scores)  # 25% threshold for the current question
      filtered_answer_indices = [i for i, score in enumerate(normalized_scores) if score >= threshold_value]
      filtered_answer_indices_all.append((query_index, filtered_answer_indices))

    # map to the nested structure to get the actual answers texts
    related_answers = []
    filtered_scores = []
    for idx, filtered_indices in filtered_answer_indices_all:
        related_answers.extend([dataset_dict['answers'][idx][i] for i in filtered_indices])
        filtered_scores.extend([dataset_dict['score'][idx][i] for i in filtered_indices])

    return related_answers, filtered_scores


In [19]:
# train_question_embeddings = encode_questions(model_bert, tokenizer_bert, train_dataset_dict["questions"], batch_size=128)
# train_answer_embeddings = encode_answers(model_bert, tokenizer_bert, train_dataset_dict["answers"], batch_size=128)

In [20]:
# save embeddings
# save_Q_encodings(train_question_embeddings, '/kaggle/working/train_question_embeddings.npy')

In [21]:
# valid_question_embeddings = encode_questions(model_bert, tokenizer_bert, valid_dataset_dict["questions"], batch_size=128)
# valid_answer_embeddings = encode_answers(model, tokenizer, valid_dataset_dict["answers"], batch_size=128)

In [22]:
# save embeddings
# save_Q_encodings(valid_question_embeddings, '/kaggle/working/valid_question_embeddings.npy')

In [23]:
# load question embeddingds created before

train_question_embeddings, valid_question_embeddings = load_Q_encodings("/content/drive/MyDrive/train_question_embeddings.npy", "/content/drive/MyDrive/valid_question_embeddings.npy")

# train_question_embeddings, valid_question_embeddings = load_Q_encodings('/kaggle/input/question-embeddings/train_question_embeddings.npy', '/kaggle/input/question-embeddings/valid_question_embeddings.npy')

Loaded Question Embeddings.


In [24]:
train_question_embeddings.to(device)
valid_question_embeddings.to(device)

tensor([[ 0.1545, -0.1431, -0.0429,  ..., -0.1290,  0.0298,  0.2459],
        [ 0.0139, -0.0602, -0.1947,  ..., -0.0839,  0.0065,  0.0863],
        [ 0.1971,  0.2178, -0.1768,  ..., -0.1064, -0.0811,  0.0184],
        ...,
        [ 0.0806,  0.1664,  0.2081,  ..., -0.1165,  0.1506,  0.1235],
        [ 0.1994,  0.0462,  0.0250,  ..., -0.0495, -0.0726,  0.1578],
        [ 0.1103,  0.0906,  0.0277,  ...,  0.0394, -0.0073,  0.0585]],
       device='cuda:0')

In [25]:
# How to find similar questions for a particulaqr query

# query = train_dataset_dict['questions'][50]
# sim_questions, sim_scores, sim_indices = retrieve_similar_questions_bert(model_bert, tokenizer_bert, train_dataset_dict, query, 50, train_question_embeddings)

# sim_questions

In [28]:
# How to find similar answers and extract context for a particulaqr query

# query = train_dataset_dict['questions'][678]
# query_index = 678
query =  'how to talk Korean?'
query_index = None
print(f'query: {query}')
sim_answers, _ = retrieve_similar_answers_bert(model_bert, tokenizer_bert, train_dataset_dict, query, query_index, train_question_embeddings, top_n_answers=8)

pd.DataFrame(sim_answers)

query: how to talk Korean?


,0
0,Korean uses an alphabet called hangul. Individ...
1,It's all context. Babies minds are sponges. Yo...
2,You would need someone helping you learn some ...
3,"Im a speech therapist, the idea behind tongue ..."
4,"If it makes you feel any better, Bono had the ..."


## Prepare the dataset for training.

Create a new column "context" by joining all the similar answers catched from dataset.

> - Keep only the top-rated answers.
>
> - To ensure that the SoS and EoS tokens are there, we have to add these tokens ourselves.

Lets create this column context for both train and valid sets.

In [29]:
from sklearn.preprocessing import MinMaxScaler

In [31]:
# for train set
train_contexts = []
train_answer = []
scaler = MinMaxScaler()
for index, q in enumerate(train_dataset_dict['questions']):
    similar_answers, _ = retrieve_similar_answers_bert(model_bert, tokenizer_bert, train_dataset_dict,
                                                    q, index, train_question_embeddings,
                                                    top_n_answers=8)

    # keep top-rated answers
    scores = np.array(train_dataset_dict['score'][index]).reshape(-1, 1)
    normalized_scores = scaler.fit_transform(scores).flatten()
    threshold = 0.25 * np.max(normalized_scores)
    filtered_indices = [i for i, score in enumerate(normalized_scores) if score >= threshold]

#     train_dataset_dict['score'] = [train_dataset_dict['score'][index][i] for i in filtered_indices]
#     train_dataset_dict['answers'] = [train_dataset_dict['answers'][index][i] for i in filtered_indices]

    # for answers -> concatenate Sos & EoS tokens to each answer and then join them into one string
    formatted_answers = ['<s> ' + answer + ' </s>' for answer in train_dataset_dict['answers'][index]]
    combined_answers = ' '.join(formatted_answers)
    train_answer.append(combined_answers)

    # for contexts -> concatenate Sos & EoS tokens to each answer and then join them into one string
    formatted_answers = ['<s> ' + answer + ' </s>' for answer in similar_answers]
    combined_answers = ' '.join(formatted_answers)
    train_contexts.append(combined_answers)

# add a new feature to dataset_dict
train_dataset_dict['contexts'] = train_contexts
train_dataset_dict['one_answer'] = train_answer

In [ ]:
# for validation set
valid_contexts = []
valid_answer = []
for index, q in enumerate(valid_dataset_dict['questions']):
    similar_answers, _ = retrieve_similar_answers_bert(model_bert, tokenizer_bert, valid_dataset_dict,
                                                    q, index, valid_question_embeddings,
                                                    top_n_answers=8)

    # keep top-rated answers
    scores = np.array(valid_dataset_dict['score'][index]).reshape(-1, 1)
    normalized_scores = scaler.fit_transform(scores).flatten()
    threshold = 0.25 * np.max(normalized_scores)
    filtered_indices = [i for i, score in enumerate(normalized_scores) if score >= threshold]

#     train_dataset_dict['score'] = [train_dataset_dict['score'][index][i] for i in filtered_indices]
#     train_dataset_dict['answers'] = [train_dataset_dict['answers'][index][i] for i in filtered_indices]

    # for answers -> concatenate Sos & EoS tokens to each answer and then join them into one string
    formatted_answers = ['<s> ' + answer + ' </s>' for answer in valid_dataset_dict['answers'][index]]
    combined_answers = ' '.join(formatted_answers)
    valid_answer.append(combined_answers)

    # for contexts -> concatenate Sos & EoS tokens to each answer and then join them into one string
    formatted_answers = ['<s> ' + answer + ' </s>' for answer in similar_answers]
    combined_answers = ' '.join(formatted_answers)
    valid_contexts.append(combined_answers)

# add a new feature to dataset_dict
valid_dataset_dict['contexts'] = valid_contexts
valid_dataset_dict['one_answer'] = valid_answer

## Train T5 for generative part

Instead of creating a summery for each question, lets train T5 model with the contexts be these answers found as similar answers and the answer be the real answer of question.

In [32]:
!pip install evaluate
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=8036dbeaef99327ac3633c83c37f4196681c276e9c2ef8aa7831fe4192227e21
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [33]:
from datasets import Dataset
import nltk
import evaluate

In [34]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

   return result

In [ ]:
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_name = 't5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

model.to('cuda')

def preprocess_data_tokenization(examples):
  inputs = [f'question: {q} , context: {c}' for q,c in zip(examples['questions'], examples['contexts'])]
  model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding='max_length')

  # Our target will be the answers
  with tokenizer.as_target_tokenizer():
      labels = tokenizer(examples['one_answer'], max_length=128, truncation=True, padding='max_length')

  model_inputs['labels'] = labels['input_ids']
  return model_inputs

# Convert lists to Dataset objects
t = Dataset.from_dict(train_dataset_dict)
v = Dataset.from_dict(valid_dataset_dict)
# Apply the preprocess function
train_dataset = t.map(preprocess_data_tokenization, batched=True)
val_dataset = v.map(preprocess_data_tokenization, batched=True)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map:   0%|          | 0/91772 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:309: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/5446 [00:00<?, ? examples/s]

In [ ]:
# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 8
PER_DEVICE_EVAL_BATCH = 8
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 2

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="/kaggle/working/",
   evaluation_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False
)

trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=train_dataset,
   eval_dataset=val_dataset,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
import os
os.environ["WANDB_DISABLED"] = 'true'

In [ ]:

trainer.train()

wandb: Tracking run with wandb version 0.16.6
wandb: W&B syncing is set to `offline` in this directory.  
wandb: Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,1.795400,1.511625,0.220834,0.171338,0.212518,0.217883
2,1.694400,1.494133,0.222924,0.173443,0.214322,0.219918


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=22944, training_loss=1.777142244070311, metrics={'train_runtime': 8542.6736, 'train_samples_per_second': 21.486, 'train_steps_per_second': 2.686, 'total_flos': 2.794264176623616e+16, 'train_loss': 1.777142244070311, 'epoch': 2.0})

In [ ]:
# save model and tokenizer
trainer.save_model("/kaggle/working/t5-with-context-finetuned")
tokenizer.save_pretrained("/kaggle/working/t5-with-context-finetuned")

('/kaggle/working/t5-with-context-finetuned/tokenizer_config.json',
 '/kaggle/working/t5-with-context-finetuned/special_tokens_map.json',
 '/kaggle/working/t5-with-context-finetuned/spiece.model',
 '/kaggle/working/t5-with-context-finetuned/added_tokens.json')

## Evaluayte the model

In [35]:
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [36]:
# load model and tokenizer
# model_dir = "/kaggle/working/t5-with-context-finetuned"
model_dir = "/content/drive/MyDrive/t5-with-context-finetuned"

# Load the model and tokenizer from the saved directory
model = T5ForConditionalGeneration.from_pretrained(model_dir)
tokenizer = T5Tokenizer.from_pretrained(model_dir)

# device = torch.device('cuda')
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

Now lets try a new question. what we do is that try to find a context based on similar questions in our dataset and then generate an asnwer with that context using the trained model.

In [37]:
def retrieve_contexts(model, tokenizer, dataset_dict, question, train_question_embeddings, index=None, top_n=5):
    similar_answers, _ = retrieve_similar_answers_bert(model, tokenizer, train_dataset_dict, question, index, train_question_embeddings, top_n_answers=8)
    return similar_answers

def generate_answer(question, context):
    # This function uses the T5 model to generate an answer from the provided contexts.
    input_text = "question: {} context: {}".format(question, " ".join(context))
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
    outputs = model.generate(inputs, max_length=128, num_beams=5, early_stopping=True).to(device)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [45]:
# Example usage
question = train_dataset_dict['questions'][50]
answer = weighted_sample(train_dataset_dict['answers'][50], train_dataset_dict['score'][50], k=1)
# question = valid_dataset_dict['questions'][8]
context = retrieve_contexts(model_bert, tokenizer_bert, train_dataset_dict, question, train_question_embeddings, index=None, top_n=8)
# print(context)
generated_answer = generate_answer(question, context)



In [55]:
print(f'Question: {question}')
print('-------------------------')
print(f'Top answer: {answer}')
print('-------------------------')
print(f'Context: {context}')
print('-------------------------')
print(f'Generated answer: {generated_answer}')

Question: Why do most spiders have many eyes, but poor vision?
-------------------------
Top answer: ['Predatory [jumping spiders]( URL_0 ) actually have quite good vision, but that doesn\'t necessarily answer your question of why *other* species of spiders have so many eyes if they don\'t get good vision out of it. The "secondary eyes" apparently mostly detect the direction of light and/or movement, which is presumably useful enough that they don\'t atrophy completely over evolutionary time (some species that live in dark caves have no eyes at all). Having enough secondary eyes that they can monitor a 360° arc would then be useful.']
-------------------------
Context: ['Tens of thousands of years of people dying to spider bites weeded out those who did not fear them, and left those who fear them as the survivors, thus entrenching that fear in us genetically.', "Hammerhead sharks apparently have significant overlap in their field of vision which makes their ability to track prey with b

In [63]:
# Example usage
index = 20
question = valid_dataset_dict['questions'][index]
answer = weighted_sample(valid_dataset_dict['answers'][index], valid_dataset_dict['score'][index], k=1)
context = retrieve_contexts(model_bert, tokenizer_bert, train_dataset_dict, question, train_question_embeddings, index=None, top_n=8)
generated_answer = generate_answer(question, context)



In [64]:
print(f'Question: {question}')
print('-------------------------')
print(f'Top answer: {answer}')
print('-------------------------')
print(f'Context: {context}')
print('-------------------------')
print(f'Generated answer: {generated_answer}')

Question: Why does so much of American youth culture originate from the African-American culture?
-------------------------
Top answer: ["Probably because while in the minority it must be admitted that the music tends to be quite dynamic. Young people don't want to be bored and tend to gravitate towards new experiences whether good or bad."]
-------------------------
Context: ['Many of the competitive industrial players are in the West so the best and brightest Asian students are drawn there. Likewise, a huge majority of scientific research and funding is in the West, so their scientists and STEM people are drawn there. However, the number of power global firms based in Asia is growing and the advances they have made in technology over the last 10-20 years is astounding. You say that much of the technological advance comes from there, but look at what Asian countries have done for consumer electronics (LG, Samsung, etc), Tech commerce (Alibaba), energy, banking, and transportation. All

In [69]:
# Example usage
index = 88
question = test_dataset_dict['questions'][index]
answer = weighted_sample(test_dataset_dict['answers'][index], test_dataset_dict['score'][index], k=1)
context = retrieve_contexts(model_bert, tokenizer_bert, train_dataset_dict, question, train_question_embeddings, index=None, top_n=8)
generated_answer = generate_answer(question, context)



In [70]:
print(f'Question: {question}')
print('-------------------------')
print(f'Top answer: {answer}')
print('-------------------------')
print(f'Context: {context}')
print('-------------------------')
print(f'Generated answer: {generated_answer}')

Question: How text messages sent to a dead phone will arrive once the phone is turned on
-------------------------
Top answer: ['The text is sent to a server belonging to your service provider and is stored, when the phone turns back on it sends the server a signal saying its on and the server sends all the texts that were received and stored while the phone was off.']
-------------------------
Context: ["You're likely not calling their real number, as they've spoofed the number that shows up on your caller ID. The new pattern by these telemarketers and fraudsters is to replace the calling number to something that you'd be more likely to answer from, like a similar number in your area.", "If the battery is run completely dead, it can't be recharged. The phone manufacturer knows this, and programs the phone to shut off before it gets ruined. When you restart it without recharging it first, it takes a little time for the circuitry to detect this and shut off itself to protect the battery

## Generate Answer

In [74]:
def answer_query(model, tokenizer, query,temperature=0.6,long_answer = False):

    context = retrieve_contexts(model_bert, tokenizer_bert, train_dataset_dict, question, train_question_embeddings, index=None, top_n=8)
    prompt = "question: {} context: {}".format(question, " ".join(context))

    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()

    if long_answer:

        outputs = model.generate(input_ids=input_ids,


                             min_length=100,
                             max_new_tokens=300,
                                 #repetition_penalty=0.2,
                             do_sample=True,
                            #  top_p=0.9,
                             temperature=temperature)

    else:

        outputs = model.generate(input_ids=input_ids,


                             min_length=30,
                             max_length = 100,
                             max_new_tokens=70,
                                 #repetition_penalty=0.2,
                             do_sample=True,
                            #  top_p=0.9,
                             temperature=temperature)
    answer = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

    return answer

In [72]:
# Remove question from answer and remove last uncomplete sentence
def pretty_asnwer(answer):
    new_answer = answer
    if new_answer.endswith('.') or new_answer.endswith('!') or new_answer.endswith('?'):

        last_index_of_question = len(prompt) +41
        new_answer = new_answer[last_index_of_question:]
        if new_answer.find('Answer')>-1:
          index = new_answer.find('Answer')
          new_answer = new_answer[index+8:]
        #print(new_answer)

    else:
        #print('Not complete answer')
        last_index_of_question = len(prompt) +41
        new_answer = new_answer[last_index_of_question:]
        last_sentence = new_answer.split('. ')[-1]
        new_answer = new_answer.replace(last_sentence, " ")
        if new_answer.find('Answer')>-1:
          #print('here')
          index = new_answer.find('Answer')
          new_answer = new_answer[index+8:]



    return new_answer

In [75]:
number_of_questions_to_get_score = 100
generated_answer_list = []

disablity_of_model = 0

for prompt in tqdm(test_dataset_dict['questions'][:number_of_questions_to_get_score]):
  #print('index: ', index)

  answer = answer_query(model, tokenizer, prompt,long_answer=True)  # Generate Answer
  new_answer = pretty_asnwer(answer) # Make it beatiful

  j=0
  a = 0
  while len(new_answer)<5: # Generate again if, last time model was disable
      a += 1
      #print(f'index_{index}_J{j}:')
      disablity_of_model+=1
      answer = answer_query(model, tokenizer, prompt,long_answer=True)
      new_answer = pretty_asnwer(answer)
      if (a > 2):
        break
      #print('new_answer: ',new_answer)

  generated_answer_list.append(new_answer) # Store answer


# Create a dataframe with orginal questions and long genereated answers
dictionary = {'questions':test_dataset_dict['questions'][:number_of_questions_to_get_score],'Generated_answers':generated_answer_list}
generated_dataset = pd.DataFrame(dictionary)
print('disablity_of_model: ',disablity_of_model)

100%|██████████| 100/100 [04:47<00:00,  2.88s/it]

disablity_of_model:  7


In [78]:
# Add True answers to dataset
concatenaed_answer_list = []
for answer_list in test_dataset_dict['answers'][:number_of_questions_to_get_score]:
  concatenaed_answer = ' '.join(answer_list)
  concatenaed_answer_list.append(concatenaed_answer)

generated_dataset['True_answers'] = concatenaed_answer_list
generated_dataset

,questions,Generated_answers,True_answers
0,why does stretching feel good?,"e, so that the phone knows that it is not dead...","as u/Midnight Rising said, you use your muscle..."
1,Why blueprints are done on blue paper,he other calls or messages that are not answer...,The drawings (before computers) were done on a...
2,What is the significance of prime numbers in e...,"ly sends the message to the device, and the ph...","If you take one number, let's say five, and mu..."
3,How are fireworks able to make certain shapes ...,"ery, it may not be able to run. If it's a batt...",Because the secondary charge is packed in the ...
4,What exactly does it mean for a system to be 8...,t sends a message to the phone's phone's phone...,A lot of replies in this thread say that it is...
...,...,...,...
95,What is a Denial of Service (DoS) attack.,ical circuit that is designed to take your pho...,"As some internet smartass once put it ""Wanna D..."
96,How are devices like cellphones and laptops st...,"t not know what it's doing, or it might not ha...",Cellphones are a bad example because they'll s...
97,"Why can you sometimes ""feel"" someone watching ...","tery can't handle the load, so it doesn't need...",When you turn to look at someone they can noti...
98,Why do SCUBA divers go into the water backward...,one with built-in software on it. If you're on...,"If they don't go back first, they go feet firs..."


## Compute Metrics

In [80]:
from rouge_score import rouge_scorer
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_rouge(predicted, reference):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
    scores = scorer.score(reference, predicted)
    scores = {metric: {sub_metric: round(value * 100, 2) for sub_metric, value in scores[metric]._asdict().items()} for metric in scores}

    return scores



rouge_list = []
rouge1_scores = {'precision': [], 'recall': [], 'fmeasure': []}
rouge2_scores = {'precision': [], 'recall': [], 'fmeasure': []}
rougeL_scores = {'precision': [], 'recall': [], 'fmeasure': []}

for index in range(generated_dataset.shape[0]):
  predicted_text = generated_dataset.iloc[index,1]
  reference_text = generated_dataset.iloc[index,2]

  rouge_scores = calculate_rouge(predicted_text, reference_text)
  rouge_list.append(rouge_scores)

  for sub_metric in rouge1_scores:
        rouge1_scores[sub_metric].append(rouge_scores['rouge1'][sub_metric])
        rouge2_scores[sub_metric].append(rouge_scores['rouge2'][sub_metric])
        rougeL_scores[sub_metric].append(rouge_scores['rougeL'][sub_metric])


generated_dataset['Rouge Scores'] = rouge_list


#mean
mean_rouge1_scores = {sub_metric: round(sum(rouge1_scores[sub_metric]) / len(rouge1_scores[sub_metric]), 2) for sub_metric in rouge1_scores}
mean_rouge2_scores = {sub_metric: round(sum(rouge2_scores[sub_metric]) / len(rouge2_scores[sub_metric]), 2) for sub_metric in rouge2_scores}
mean_rougeL_scores = {sub_metric: round(sum(rougeL_scores[sub_metric]) / len(rougeL_scores[sub_metric]), 2) for sub_metric in rougeL_scores}
print('For 100 items of test dataset:')
print('*'*20)
print("Mean ROUGE-1 scores (as percentages):", mean_rouge1_scores)
print("Mean ROUGE-2 scores (as percentages):", mean_rouge2_scores)
print("Mean ROUGE-L scores (as percentages):", mean_rougeL_scores)

For 100 items of test dataset:
********************
Mean ROUGE-1 scores (as percentages): {'precision': 36.54, 'recall': 12.51, 'fmeasure': 15.91}
Mean ROUGE-2 scores (as percentages): {'precision': 3.77, 'recall': 1.01, 'fmeasure': 1.3}
Mean ROUGE-L scores (as percentages): {'precision': 24.52, 'recall': 8.23, 'fmeasure': 10.39}


In [82]:
import json
# Save dataframe
generated_dataset.to_csv(f"/content/drive/MyDrive/generated_{number_of_questions_to_get_score}_test_dataset.csv",sep=',',index=False)
# Svae mean scores
# Filename
filename = f"/content/drive/MyDrive/mean_rouge_scores_for_{number_of_questions_to_get_score}_test_dataset.txt"
note = f'These valeus are for train_dataset of {number_of_questions_to_get_score} questions '
# Writing to the text file
with open(filename, 'w') as file:
    file.write(note + "\n\n")

    file.write("Mean ROUGE-1 scores:\n")
    file.write(json.dumps(mean_rouge1_scores, indent=4) + "\n\n")

    file.write("Mean ROUGE-2 scores:\n")
    file.write(json.dumps(mean_rouge2_scores, indent=4) + "\n\n")

    file.write("Mean ROUGE-L scores:\n")
    file.write(json.dumps(mean_rougeL_scores, indent=4) + "\n")

print(f"Data saved to {filename}")

Data saved to /content/drive/MyDrive/mean_rouge_scores_for_100_test_dataset.txt


As it is clear in above results, the model is working better if the context retrieved after searching the dataset is reletive to the question asked.

So in next step, we are going to change the embedding and the searching for top-k similar questions and see if it improves the model.